In [0]:
# Importo los elementos necesarios para la tarea
from pyspark.sql.types import *
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
import pandas as pd
import pyspark.sql.functions as F

# Creación dataframe ratings
ratings_schema = StructType(
  [StructField('userId', IntegerType()),
   StructField('movieId', IntegerType()),
   StructField('rating', DoubleType())])

ratingsDF=spark.read.option("header","false").option("delimiter",";").option("inferSchema","true")\
                          .schema(ratings_schema).csv("/FileStore/tables/udata.csv")

ratingsDF.show(10)

+------+-------+------+
userId|movieId|rating|
+------+-------+------+
 196| 242| 3.0|
 186| 302| 3.0|
 22| 377| 1.0|
 244| 51| 2.0|
 166| 346| 1.0|
 298| 474| 4.0|
 115| 265| 2.0|
 253| 465| 5.0|
 305| 451| 3.0|
 6| 86| 3.0|
+------+-------+------+
only showing top 10 rows

In [0]:
# Creación dataframe películas

pelis_schema = StructType(
  [StructField('ID', IntegerType()),
   StructField('title', StringType())])

peliculasDF=spark.read.option("header","false").option("delimiter",";").option("inferSchema","true")\
                          .schema(pelis_schema).csv("/FileStore/tables/uitem.csv")
peliculasDF.show(10)

+---+--------------------+
 ID| title|
+---+--------------------+
 1| Toy Story (1995)|
 2| GoldenEye (1995)|
 3| Four Rooms (1995)|
 4| Get Shorty (1995)|
 5| Copycat (1995)|
 6|Shanghai Triad (Y...|
 7|Twelve Monkeys (1...|
 8| Babe (1995)|
 9|Dead Man Walking ...|
 10| Richard III (1995)|
+---+--------------------+
only showing top 10 rows

In [0]:
# Hago un Inner Join de los dataframes películas y ratings (ya que comparten los valores de movieId) para ver el título de las películas y entender mejor la información y las predicciones

movielensDF = ratingsDF.join(peliculasDF, ratingsDF.movieId == peliculasDF.ID).select("userId","movieId","title","rating")
movielensDF.show(8)


+------+-------+--------------------+------+
userId|movieId| title|rating|
+------+-------+--------------------+------+
 196| 242| Kolya (1996)| 3.0|
 186| 302|L.A. Confidential...| 3.0|
 22| 377| Heavyweights (1994)| 1.0|
 244| 51|Legends of the Fa...| 2.0|
 166| 346| Jackie Brown (1997)| 1.0|
 298| 474|Dr. Strangelove o...| 4.0|
 115| 265|Hunt for Red Octo...| 2.0|
 253| 465|Jungle Book, The ...| 5.0|
+------+-------+--------------------+------+
only showing top 8 rows

In [0]:
# Creación del modelo predictivo con ALS. 

# Creo sets de train y test
(training_data, test_data) = movielensDF.randomSplit([0.8, 0.2])

# Construyo el modelo a partir del algoritmo ALS

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative=True,
coldStartStrategy="drop", implicitPrefs=False)

# Definición del paramGridbuilder

param_grid=ParamGridBuilder()\
           .addGrid(als.rank,[12,13,14])\
           .addGrid(als.maxIter,[18,19,20])\
           .addGrid(als.regParam, [.17,.18,.19])\
           .build()     

# Definición como Evaluator del RMSE

evaluator= RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
                    
# Cross validation con Train validation split

tvs= TrainValidationSplit(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator)
                    
# Aplicamos el modelo al subset de training
model = tvs.fit(training_data)

# Definición del Mejor modelo
mejor_modelo=model.bestModel

# Basandondos en ese mejor modelo encontrado generamos las predicciones para la muestra de test (test_data)
predictions = mejor_modelo.transform(test_data)

# Imprimimos el resultado final del RMSE resultante con los mejores parametros del modelo
rmse = evaluator.evaluate(predictions)
print ("RMSE: "), rmse
print("Con estos parámetros, el mejor modelo es:")
print("           Rank:"),mejor_modelo.rank
print("           MaxIter:"),mejor_modelo._java_obj.parent().getMaxIter()
print("           RegParam:"),mejor_modelo._java_obj.parent().getRegParam()


RMSE: 0.920267958862
Con estos parámetros, el mejor modelo es:
 Rank: 13
 MaxIter: 20
 RegParam: 0.17

In [0]:
# Mejor resultado RMSE=  Tardó unos 40 minutos (137 jobs)
RMSE:  0.920267958862
# Con estos parámetros, el mejor modelo es:
           Rank: 13 (latent factors)
           MaxIter: 20
           RegParam: 0.17

In [0]:
# Obtengo unas cuantas predicciones y las comparo con la puntuación real dada por los usuarios:

predictions.show(10) 

+------+-------+--------------------+------+----------+
userId|movieId| title|rating|prediction|
+------+-------+--------------------+------+----------+
 26| 148|Ghost and the Dar...| 3.0| 2.593676|
 27| 148|Ghost and the Dar...| 3.0| 3.0213172|
 606| 148|Ghost and the Dar...| 3.0| 3.4664779|
 416| 148|Ghost and the Dar...| 5.0| 3.6172678|
 935| 148|Ghost and the Dar...| 4.0| 3.8481612|
 328| 148|Ghost and the Dar...| 3.0| 3.0478349|
 923| 148|Ghost and the Dar...| 4.0| 3.6771605|
 269| 148|Ghost and the Dar...| 1.0| 1.6652021|
 486| 148|Ghost and the Dar...| 2.0| 2.521703|
 627| 148|Ghost and the Dar...| 3.0| 2.735921|
+------+-------+--------------------+------+----------+
only showing top 10 rows

In [0]:
# Comparamos las predicciones con los ratings para cada usuario mejorando la tabla visualmente con la opción display

display(predictions.sort("userId","rating"))

userId,movieId,title,rating,prediction
1,247,Turbo: A Power Rangers Movie (1997),1.0,1.1001503
1,260,Event Horizon (1997),1.0,2.3963456
1,140,Homeward Bound: The Incredible Journey (1993),1.0,2.504637
1,112,Flipper (1996),1.0,2.081625
1,237,Jerry Maguire (1996),2.0,3.5295262
1,167,Private Benjamin (1980),2.0,2.7352147
1,149,Jude (1996),2.0,3.2702394
1,101,Heavy Metal (1981),2.0,3.2997768
1,126,"Spitfire Grill, The (1996)",2.0,3.8093193
1,233,Under Siege (1992),2.0,3.0048223


In [0]:
# Tras investigar las posibilidades que ofrece ALS en pyspark.ml.recommendation module, voy a poner en práctica varias funciones que ofrecen recomendaciones de películas personalizadas para todos los usuarios y que también recomiendan usarios a los que probablemente más les gustaría una película concreta, como son recommendForAllUsers() y recommendForAllItems()

# Ofrecemos por ejemplo 2 recomendaciones de peliculas por usuario. Junto al Id de cada usario se ofrece una lista con los 2 movieIds de peliculas recomendadas y su predicción

recomusers=mejor_modelo.recommendForAllUsers(2).toPandas()
recomusers


Out[ 37 ]: 
 userId recommendations
0 471 [(1612, 4.91252279282), (680, 4.43761491776)]
1 463 [(1642, 4.03397035599), (1449, 3.96559262276)]
2 833 [(1368, 4.37402963638), (320, 4.32030677795)]
3 496 [(1467, 3.95330262184), (1642, 3.87397289276)]
4 148 [(1467, 5.08030509949), (1449, 5.00364112854)]
5 540 [(1642, 4.83469343185), (1467, 4.81271123886)]
6 392 [(1449, 5.13690662384), (1398, 4.9149851799)]
7 243 [(1449, 4.63963365555), (1642, 4.62353754044)]
8 623 [(1599, 4.32598829269), (318, 4.3085193634)]
9 737 [(1449, 4.71138238907), (1368, 4.68730068207)]
10 897 [(1467, 5.1715388298), (1344, 4.85869884491)]
11 858 [(1449, 4.47133111954), (1242, 4.43227529526)]
12 31 [(1536, 4.88390302658), (1368, 4.85520410538)]
13 516 [(1449, 5.01841545105), (1642, 4.99501085281)]
14 580 [(1467, 4.51982498169), (1642, 4.36119651794)]
15 251 [(1467, 5.00329971313), (1449, 4.78702783585)]
16 451 [(1344, 4.55449008942), (1467, 4.09443426132)]
17 85 [(1449, 4.40883922577), (1467, 4.28567266464)]
18 137 [(1467, 5.49991846085), (1500, 5.4231133461)]
19 808 [(1449, 5.38765668869), (1467, 5.36210346222)]
20 883 [(1642, 4.95862913132), (1449, 4.80601024628)]
21 65 [(1467, 4.69930267334), (1449, 4.55874061584)]
22 458 [(1449, 4.6729388237), (1642, 4.58009767532)]
23 879 [(1467, 4.70070362091), (1642, 4.45472955704)]
24 481 [(1449, 4.77241611481), (1642, 4.69031524658)]
25 53 [(1467, 4.69067239761), (1449, 4.53185510635)]
26 804 [(1467, 4.81922626495), (1344, 4.67505931854)]
27 255 [(1467, 3.9020755291), (838, 3.69550681114)]
28 588 [(1642, 4.79587030411), (1467, 4.76381158829)]
29 898 [(1642, 4.80105781555), (1398, 4.65013217926)]
.. ... ...
913 74 [(1467, 4.76921892166), (1449, 4.60901784897)]
914 934 [(1467, 4.56427097321), (1449, 4.5147228241)]
915 755 [(1344, 4.31968355179), (1233, 4.13437986374)]
916 138 [(1467, 5.18194580078), (1449, 5.07688236237)]
917 180 [(1642, 5.09698390961), (1449, 4.91440534592)]
918 932 [(1449, 5.0461359024), (1642, 4.87511396408)]
919 483 [(1467, 3.71137547493), (408, 3.66728687286)]
920 643 [(1467, 4.86560201645), (1449, 4.67811012268)]
921 104 [(1467, 3.78453278542), (1449, 3.69668984413)]
922 656 [(1463, 3.85563158989), (1368, 3.62925124168)]
923 917 [(1642, 4.921833992), (1449, 4.73973560333)]
924 468 [(1449, 5.00747108459), (1467, 4.99246883392)]
925 841 [(1344, 4.90688514709), (1467, 4.88467216492)]
926 134 [(1344, 4.83696985245), (1467, 4.44296503067)]
927 584 [(1467, 4.56948184967), (1344, 4.53285312653)]
928 878 [(1449, 4.06208372116), (1398, 3.95364689827)]
929 521 [(1467, 4.17841100693), (1500, 3.80311775208)]
930 407 [(1467, 4.51868104935), (1449, 4.32112264633)]
931 788 [(1449, 4.2445473671), (1467, 4.23527860641)]
932 701 [(1642, 5.1460351944), (1449, 5.1353096962)]
933 617 [(320, 3.89997053146), (1368, 3.88836550713)]
934 393 [(1344, 4.4132437706), (1467, 4.24424886703)]
935 315 [(1642, 4.97269916534), (1449, 4.95293951035)]
936 36 [(1607, 5.48954963684), (1643, 5.34250831604)]
937 187 [(1449, 4.80217933655), (1467, 4.80115842819)]
938 208 [(814, 4.38991308212), (1599, 4.33227586746)]
939 401 [(1642, 3.94104838371), (1242, 3.86918473244)]
940 422 [(1467, 4.83498620987), (1642, 4.64991950989)]
941 517 [(1467, 4.20853281021), (1500, 4.08385944366)]
942 89 [(1467, 5.11324310303), (1642, 5.05502557755)]

[943 rows x 2 columns]

In [0]:
#Hago lo mismo con recommendForAllItems, función que recomienda a que usarios le puede gustar una película concreta que elijamos. En el ejemplo recomienda 2 usuarios por cada película

recomitems=mejor_modelo.recommendForAllItems(2).toPandas()
recomitems


Out[ 41 ]: 
 movieId recommendations
0 1580 [(127, 1.27756798267), (340, 1.25718259811)]
1 471 [(688, 4.83772134781), (810, 4.735496521)]
2 1591 [(34, 5.20796108246), (519, 4.82907152176)]
3 1342 [(810, 3.94625163078), (118, 3.7484228611)]
4 463 [(928, 4.55803918839), (810, 4.54694366455)]
5 833 [(688, 4.42221069336), (127, 4.31783294678)]
6 1645 [(928, 5.60216283798), (341, 5.59079313278)]
7 496 [(688, 5.35236120224), (810, 5.32640600204)]
8 148 [(507, 4.80090999603), (688, 4.7797665596)]
9 1088 [(127, 3.62506651878), (688, 3.57277464867)]
10 1238 [(519, 4.36840629578), (507, 4.0708026886)]
11 540 [(688, 3.64535307884), (507, 3.58582830429)]
12 1460 [(810, 3.94037008286), (928, 3.72091245651)]
13 392 [(688, 4.95277881622), (507, 4.85405921936)]
14 1522 [(427, 3.69603037834), (688, 3.69233322144)]
15 243 [(127, 4.12864160538), (507, 4.0323343277)]
16 623 [(688, 4.41177606583), (507, 4.35974979401)]
17 1483 [(127, 5.25760698318), (507, 4.6212220192)]
18 1084 [(810, 5.19045734406), (427, 4.80086183548)]
19 1025 [(507, 4.48202419281), (628, 4.35115242004)]
20 1395 [(118, 3.69134116173), (219, 3.65262079239)]
21 737 [(118, 3.64503860474), (810, 3.61051535606)]
22 897 [(507, 2.15750026703), (127, 2.00958609581)]
23 1127 [(688, 4.79318237305), (939, 4.73547267914)]
24 1507 [(810, 3.84387469292), (4, 3.67179322243)]
25 858 [(810, 1.37646460533), (849, 1.33010900021)]
26 1270 [(127, 3.51507401466), (507, 3.48962235451)]
27 1650 [(928, 5.60216283798), (341, 5.59079313278)]
28 31 [(507, 4.91228580475), (810, 4.80994272232)]
29 1322 [(810, 3.3927898407), (688, 3.15686798096)]
... ... ...
1622 841 [(507, 4.12738704681), (127, 3.9727268219)]
1623 991 [(427, 3.69292855263), (34, 3.61741137505)]
1624 134 [(810, 5.21259880066), (118, 5.13955879211)]
1625 584 [(810, 4.4930357933), (688, 4.32451868057)]
1626 878 [(36, 4.2610669136), (126, 4.04764556885)]
1627 521 [(810, 4.70411491394), (118, 4.5973572731)]
1628 407 [(127, 3.45861768723), (810, 3.42806339264)]
1629 788 [(427, 3.07080054283), (127, 3.04725289345)]
1630 701 [(810, 5.08794546127), (366, 5.00531673431)]
1631 963 [(810, 5.56573390961), (688, 5.29725027084)]
1632 617 [(810, 4.68335056305), (118, 4.56266832352)]
1633 1107 [(341, 4.13024759293), (928, 4.08800983429)]
1634 1387 [(688, 3.70988273621), (507, 3.64474821091)]
1635 1009 [(4, 4.10914039612), (118, 4.10745382309)]
1636 393 [(688, 4.65456676483), (810, 4.60134792328)]
1637 954 [(810, 5.00719118118), (849, 4.63062143326)]
1638 315 [(810, 5.08382892609), (688, 5.03128147125)]
1639 36 [(688, 2.93831825256), (507, 2.91729927063)]
1640 1656 [(4, 5.33052778244), (427, 5.02988052368)]
1641 187 [(810, 5.17681789398), (118, 5.02347326279)]
1642 1217 [(127, 4.26633024216), (261, 3.8897600174)]
1643 1277 [(688, 4.66320085526), (507, 4.64745473862)]
1644 208 [(810, 5.17424201965), (688, 4.86405706406)]
1645 1359 [(688, 1.97955620289), (939, 1.92258834839)]
1646 401 [(810, 3.67217803001), (261, 3.39543104172)]
1647 422 [(471, 3.70008802414), (507, 3.56135010719)]
1648 1413 [(887, 3.60712218285), (810, 3.43762922287)]
1649 517 [(810, 4.50557565689), (118, 4.45760583878)]
1650 1138 [(507, 4.84441947937), (810, 4.73359394073)]
1651 89 [(810, 5.1552772522), (118, 5.02330112457)]

[1652 rows x 2 columns]

In [0]:
# Pruebo también otras funciones explorando el dataset final y los usuarios, por ejemplo:
# Los 10 usuarios que más veces puntúan películas
usuarios_mas_activos = movielensDF.groupBy("userId").count().sort(F.col("count").desc()).limit(50000)
usuarios_mas_activos = usuarios_mas_activos.withColumnRenamed("userId", "Usuario_muy_activo")
usuarios_mas_activos.show(10)

+------------------+-----+
Usuario_muy_activo|count|
+------------------+-----+
 405| 737|
 655| 685|
 13| 636|
 450| 540|
 276| 518|
 416| 493|
 537| 490|
 303| 484|
 234| 480|
 393| 448|
+------------------+-----+
only showing top 10 rows

In [0]:
# Los 10 usuarios que más alto puntúan de media
puntuacion_media = movielensDF.groupBy("userId").agg(F.avg('rating').alias('valoracion_media')).sort(F.col("valoracion_media").desc())
puntuacion_media.show(10)

+------+-----------------+
userId| valoracion_media|
+------+-----------------+
 849|4.869565217391305|
 688|4.833333333333333|
 507|4.724137931034483|
 628|4.703703703703703|
 928| 4.6875|
 118|4.661971830985915|
 907|4.571428571428571|
 686|4.563380281690141|
 427|4.548387096774194|
 565|4.542857142857143|
+------+-----------------+
only showing top 10 rows